In [4]:
import pandas as pd
import os
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import widgets

PATH_REPO_ROOT = Path().resolve().parent
PATH_DATA = PATH_REPO_ROOT / "data"
PATH_DATA_RAW = PATH_DATA / "raw"
PATH_DATA_RAW

PosixPath('/beegfs/.global0/ws/s4610340-energy_behavior/yahor/kaggle-predict_energy_behavior_of_prosumers/data/raw')

In [3]:
df_train = pd.read_csv(PATH_DATA_RAW / "train.csv").astype({"is_consumption": bool, "is_business": bool})
df_train["datetime"] = pd.to_datetime(df_train["datetime"])
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,False,1,0.713,False,2021-09-01 00:00:00,0,0,0
1,0,False,1,96.590,True,2021-09-01 00:00:00,0,1,0
2,0,False,2,0.000,False,2021-09-01 00:00:00,0,2,1
3,0,False,2,17.314,True,2021-09-01 00:00:00,0,3,1
4,0,False,3,2.904,False,2021-09-01 00:00:00,0,4,2
...,...,...,...,...,...,...,...,...,...
2018347,15,True,0,197.233,True,2023-05-31 23:00:00,637,2018347,64
2018348,15,True,1,0.000,False,2023-05-31 23:00:00,637,2018348,59
2018349,15,True,1,28.404,True,2023-05-31 23:00:00,637,2018349,59
2018350,15,True,3,0.000,False,2023-05-31 23:00:00,637,2018350,60


In [17]:
from IPython.display import display

widget_select_unit = widgets.Dropdown(
    options=list(df_train["prediction_unit_id"].unique()),
    value=0,
    description='Unit ID:',
)
business_units = df_train.loc[df_train["is_business"]]["prediction_unit_id"].unique()
private_units = df_train.loc[~df_train["is_business"]]["prediction_unit_id"].unique()

widget_select_private = widgets.Dropdown(
    options=["private", "business"],
    value="private",
    description='Client type: '
)
trace_con = go.Scatter(name="consumption")
trace_prod = go.Scatter(name="production", opacity=0.6)
widget_container = widgets.HBox([widget_select_private, widget_select_unit])
figure_widget = go.FigureWidget(data=[trace_con, trace_prod])

def update_figure_widget(change):
    unit_series = df_train.loc[df_train["prediction_unit_id"]==widget_select_unit.value]
    unit_consumption_series = unit_series.loc[unit_series["is_consumption"]]
    unit_production_series = unit_series.loc[~unit_series["is_consumption"]]
    with figure_widget.batch_update():
        figure_widget.data[0].x = unit_consumption_series["datetime"]
        figure_widget.data[0].y = unit_consumption_series["target"]
        figure_widget.data[1].x = unit_production_series["datetime"]
        figure_widget.data[1].y = unit_production_series["target"]

def update_select_unit_values(change):
    display(widget_select_private.value)
    if widget_select_private.value=="private":
        widget_select_unit.options = private_units
    elif widget_select_private.value=="business":
        widget_select_unit.options = business_units
    
widget_select_unit.observe(update_figure_widget, "value")
widget_select_private.observe(update_select_unit_values)

display(widgets.VBox([widget_container, figure_widget]))